In [ ]:
import os
import glob
import json
import pandas as pd
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [ ]:
#Files to read
dataF = 'G:/edu/sem2/766/proj/ML_OD_Benchmarking/data/outputs/'
origF = dataF +'5000_original_results/'
tranF = dataF + 'transformed_outputs/'
algoF = 'e2e_faster_rcnn_R-50-C4_2x/'
anntF = 'G:/edu/sem2/766/proj/ML_OD_Benchmarking/data/inputs/annotations/instances_val2017.json'

imageName = '000000076547'
imageId = int(imageName)

In [ ]:
cocoGt=COCO(anntF)
#cocoDt=cocoGt.loadRes(origF+algoF + '0_e2e_faster_rcnn_R-50-C4_2x.json')

In [31]:
#imgIds = cocoGt.getImgIds()
annIds = cocoGt.getAnnIds(imgIds = [530162])
anns = cocoGt.loadAnns(annIds)
dt = pd.DataFrame(anns)
dt


,area,bbox,category_id,id,image_id,iscrowd,segmentation
0,28193.59970,"[126.72, 18.22, 381.43, 218.56]",28,283003,530162,0,"[[318.49, 26.22, 318.49, 18.78, 321.24, 18.22,..."
1,882.09260,"[0.24, 156.89, 28.77, 66.78]",28,284085,530162,0,"[[0.42, 156.89, 6.39, 160.33, 13.09, 165.58, 1..."
2,13106.33585,"[307.06, 271.55, 131.45, 154.49]",1,1230651,530162,0,"[[430.84, 420.28, 307.06, 426.04, 313.77, 366...."
3,14034.33930,"[176.56, 274.43, 134.33, 147.77]",1,1255402,530162,0,"[[308.02, 421.24, 180.4, 422.2, 176.56, 397.25..."
4,9113.10595,"[534.53, 310.58, 104.76, 115.51]",1,1277571,530162,0,"[[556.56, 379.35, 552.8, 370.75, 550.65, 365.3..."
5,7465.50055,"[0.25, 142.7, 54.64, 244.7]",1,1282917,530162,0,"[[10.13, 236.28, 10.71, 214.19, 14.78, 194.43,..."
6,16278.32565,"[139.45, 165.9, 111.56, 255.81]",1,1697952,530162,0,"[[178.88, 171.67, 165.41, 187.05, 163.49, 204...."
7,5668.68900,"[396.17, 283.41, 69.27, 143.59]",1,1700528,530162,0,"[[396.17, 319.32, 400.28, 312.65, 400.28, 303...."
8,491.92805,"[141.2, 187.23, 20.89, 37.98]",1,1702706,530162,0,"[[151.33, 225.21, 159.56, 210.65, 162.09, 212...."
9,15130.11685,"[265.8, 145.85, 151.6, 218.78]",1,1704618,530162,0,"[[293.62, 303.22, 265.8, 264.84, 270.59, 247.5..."


In [ ]:
def bbMatch(A,B, base = 'first'):
    '''Returns a Jaccard like score of match between the two bounding boxes. IntersectionArea / BaseArea'''
    areaA = A[2]*A[3]#First object's area
    areaB = B[2]*B[3]#Second objects area
    #print(areaU)
    if areaA==0 or areaB ==0:
        return 0#avoid division by zero when no match
    xa1,ya1,xa2,ya2 = A[0],A[1],A[0]+A[2], A[1]+A[3]
    xb1,yb1,xb2,yb2 = B[0],B[1],B[0]+B[2], B[1]+B[3]
    dx = min(xa2, xb2) - max(xa1, xb1)#overlap in x
    dy = min(ya2, yb2) - max(ya1, yb1)#overlap in y
    areaI= dx*dy if (dx>=0) and (dy>=0) else 0
    area = {'first':areaA,
            'second':areaB,
            'sum':areaA+areaB,
            'union': areaA+areaB-areaI,#union = sum - intersection
            'larger': (areaA if areaA>areaB else areaB),
            'smaller':(areaA if areaA<areaB else areaB)}
    return areaI/area[base];
A = [415.11, 204.57, 172.01, 258.54]
B = [423.078522, 205.677017, 580.738586, 441.871582]
bbMatch(A,B)

In [68]:
def findMatchingBBoxes(detectedJsons, cocoAnnotations,outdir):
    '''loopes over all images in detected Jsons.
    For each object in these images, finds the best matching object
    provided in coco Annotations for this particular image.
    Match is done by comparing overlapping area (Jaccard measure) of the two bounding boxes.
    Must match atleast 10%.
    Also creates a new json file that is more structured'''
    for i, js in enumerate(detectedJsons):#multiple json files returned by glob
        #if i>0:#for now to save time read only one file. Remove in actual script or parallelize
        #    break
        with open(js) as fd:
            imgs = json.load(fd)#go over each file
        outputs = [None]*len(imgs)
        for _idx,img in enumerate(imgs):#array of info
            #print(img)
            imgId = int(img['img_name'][32:-4])
            #print(imgId)
            detected = []
            count = 0
            uniques = set()
            dt = pd.DataFrame(cocoAnnotations.loadAnns(cocoGt.getAnnIds(imgIds = [imgId])))
            for index,A in enumerate(img['bboxes']):#for each object detected by NN
                #reframe information to write in json
                info ={'detClass':img['classes'][index],
                            'detBbox':A,
                            'score':img['scores'][index],
                            'matchObjectId':None,
                            'matchClass':None,
                            'matchBBox':None,}
                detected.append(info)
                if img['scores'][index]<0.5:#skip objects with low confidence
                    continue
                count = count+1
                best, bestM = None,-1
                for _, obj in dt.iterrows():#go over all given annotations for this image
                    #print(A)
                    #print(obj.bbox)
                    #Use larger as base of match, for better result as prevents saturation (match=1)
                    m = bbMatch(A,obj.bbox, 'larger')#find match between two objects' bboxes
                    if m >=0.10 and m>bestM:#best of all matches with atleast 10% match
                        best,bestM = obj,m#id defined by coco annotations
                if not best is None:#preserve order
                    info['matchObjectId']=best.id
                    info['matchClass']=best.category_id
                    info['matchBBox']=best.bbox
                    uniques.add(best.id)
            #matchedBBoxes[str(imgId)] = matches
            outputs[_idx] = {'image_id':imgId,
                             'objects':detected,#all objects detected in this image
                            'totalDetected': len(img['bboxes']),
                            'matched':count,
                            'uniqueMatches':len(uniques)}
        with open(outdir+os.path.basename(js)[:-5]+'_matches.json','w') as fd:
            fd.write(json.dumps(outputs))
    return outputs
origJsons = glob.iglob(origF+algoF + '*.json')
tranJsons = glob.iglob(tranF+algoF + '*.json')
outDir = origF+algoF[:-1]+'_matches/'#MANUALLY CREATE THIS DIRECTORY, DIDNT ADD SCRIPT TO CREATE THIS YET
matches = findMatchingBBoxes(origJsons, cocoGt, outDir)
#matches

In [56]:
a = set()
a.add(1)
a.add(1)
a

{1}